# Functions to create and manipulate an LCI database

In [3]:
def add_production_to_dataset(ds_name, product, device, inventory):
    production = {'name': ds_name,
                  'reference product': inventory.iloc[product]['reference product'],
                  'location': inventory.iloc[product]['location'],
                  'production amount': inventory.iloc[product][device],
                  'unit': inventory.iloc[product]['unit'],
                  'database': inventory.iloc[product]['database'],
                  'code': wurst.filesystem.get_uuid()
                  }
    return production

def add_production_to_exchanges(ds_dict):
    production = {'name': ds_dict['name'],
                  'reference product': ds_dict['reference product'],
                  'location': ds_dict['location'],
                  'amount': ds_dict['production amount'],
                  'unit': ds_dict['unit'],
                  'database': ds_dict['database'],
                  'type': 'production'
                 }
    return production

def add_technosphere_to_exchanges(exc, exc_amount, inventory, uncertainty=False, lower_bound=0, upper_bound=0):
    technosphere_ex = {'name': inventory.iloc[exc]['name'],
                       'reference product': inventory.iloc[exc]['reference product'],
                       'amount': exc_amount,
                       'unit': inventory.iloc[exc]['unit'],
                       'database': inventory.iloc[exc]['database'],
                       'location': inventory.iloc[exc]['location'],
                       'type': inventory.iloc[exc]['type']
                       }
    if uncertainty is True:
        technosphere_ex.update({'uncertainty type': 5,
                                'loc': exc_amount,
                                'minimum': lower_bound,
                                'maximum': upper_bound
                               })
    return technosphere_ex

def add_triangular_distribution_to_exchage(exc_range):
    uncertainty = {'uncertainty type': 5,
                   'loc': exc_range[0],
                   'minimum': exc_range[1],
                   'maximum': exc_range[2]
                  }
    return uncertainty

In [3]:
def add_key_to_exchanges(db, external_db, biosphere_db):
    '''
    This function adds the input key (database and code) to exchanges within the datasets
    Returns a dictionary with the linked dataset
    '''
    for ds in db:
        for exc in ds["exchanges"]:
            if exc['type'] in ['production', 'technosphere']:
                exc_link = wurst.get_one(db + external_db, wurst.equals("name", exc['name']),
                                                           wurst.equals("reference product", exc['product']),
                                                           wurst.equals("location", exc['location'])
                                        )
                exc.update({'input': (exc_link['database'], exc_link['code'])})
            
            elif exc['type'] == 'biosphere':
                if 'input' not in exc:
                    ef_code = [ef['code'] for ef in biosphere_db if ef['name'] == exc['name'] and 
                                                                    ef['unit'] == exc['unit'] and 
                                                                    ef['categories'] == exc['categories']][0]
                    exc.update({'input': ('biosphere3', ef_code)})      
    return db

In [15]:
def correct_reference_product_exchanges(db):
    '''
    The Ecoinvent database imported in BW2 uses 'product' instead of 'reference product' for exchanges.
    However, the ExcelImporter requires that the 'reference product' is defined for exchanges.
    This function substitute 'reference product' by 'product' accross all the datasets in a database (list of dictionaries)
    '''    
    for ds in db:
        for exc in ds["exchanges"]:
            if 'reference product' in exc:
                exc['product'] = exc.pop('reference product')
    return db

In [14]:
def create_inventories_from_df(INVENTORIES_DF, DATA_TRAFFIC):
    '''
    This function creates new inventories based on information from a dataframe
    '''
    COL_START = 7 # Inventory data start at column 7 in the dataframe
    COL_NUMBER = len(INVENTORIES_DF.columns) - COL_START - 1 # Number of columns with inventories (rest 1 to substract the 'comment' column)
    
    inventories = []
    for col in range(COL_NUMBER):
        col_no = COL_START + col
        inventory = INVENTORIES_DF.iloc[:, np.r_[0:COL_START, col_no]].copy()
        device = inventory.columns[COL_START]
    
        for product in inventory[(inventory['type'] == 'production') & inventory[device] == 1].index:
            activity = inventory.iloc[product]['name']
            ds_name = f"{activity}, {device}"
            
            # Create dataset dictionary with dataset information
            ds_dict = {'name': ds_name,
                       'reference product': inventory.iloc[product]['reference product'],
                       'location': inventory.iloc[product]['location'],
                       'production amount': inventory.iloc[product][device],
                       'unit': inventory.iloc[product]['unit'],
                       'database': inventory.iloc[product]['database'],
                       'code': wurst.filesystem.get_uuid()
                       }
            
            # Add exchanges
            exchanges = []
            
            # Add production to exchanges
            exchanges.append({'name': ds_dict['name'],
                              'product': ds_dict['reference product'],
                              'location': ds_dict['location'],
                              'amount': ds_dict['production amount'],
                              'unit': ds_dict['unit'],
                              'database': ds_dict['database'],
                              'type': 'production'
                             })
        
            # Add technosphere exchanges
            for exc in inventory[(inventory['type'] == 'technosphere') & inventory[device] != 0].index:
                # Check uncertainty
                if inventory.iloc[exc][device] == 'Data_Traffic':
                    uncertainty = True
                    exc_amount = DATA_TRAFFIC.loc[activity]['average']
                    lower_bound = DATA_TRAFFIC.loc[activity]['minimum']
                    upper_bound = DATA_TRAFFIC.loc[activity]['maximum']
                else:
                    if type(inventory.iloc[exc][device]) == str:
                        uncertainty = True
                        list_values = [float(k) for k in inventory.iloc[exc][device].lstrip("[").rstrip("]").split(',')]
                        exc_amount = list_values[0]
                        lower_bound = list_values[1]
                        upper_bound = list_values[2]
                    else:
                        uncertainty = False
                        exc_amount = inventory.iloc[exc][device]
                
                technosphere_ex = {'name': inventory.iloc[exc]['name'],
                                   'product': inventory.iloc[exc]['reference product'],
                                   'amount': exc_amount,
                                   'unit': inventory.iloc[exc]['unit'],
                                   'database': inventory.iloc[exc]['database'],
                                   'location': inventory.iloc[exc]['location'],
                                   'type': inventory.iloc[exc]['type']
                                  }
                # Update uncertainty
                if uncertainty is True:
                    technosphere_ex.update({'uncertainty type': 5,
                                            'loc': exc_amount,
                                            'minimum': lower_bound,
                                            'maximum': upper_bound
                                           })

                exchanges.append(technosphere_ex)
                
            ds_dict.update({'exchanges': exchanges})
            inventories.append(ds_dict)
    
    return inventories

In [17]:
def create_regional_inventories(ds_to_regionalize, new_locations, ei_db):
    '''
    This function create regional copies of GLO inventories and relink exchanges to the new locations
    '''
    
    # Create copies of datasets to the new locations
    ds_copy_list = []
    for ds in ds_to_regionalize:
        for loc in new_locations:
            ds_copy = wurst.transformations.geo.copy_to_new_location(ds, loc)
            ds_copy_list.append(ds_copy)

    # Relink technosphere exchanges to the new locations
    for ds in ds_copy_list:
        for exc in ds['exchanges']:
            # Relink electricity market
            if 'electricity, low voltage' in exc['product']:
                if exc['location'] != ds['location']:
                    electricity_loc = [ds_i for ds_i in ei_db if ds_i['name'] in ['market for electricity, low voltage', 'market group for electricity, low voltage'] and ds_i['location'] == ds['location']][0]
                    exc.update({'name': electricity_loc['name'],
                                'location': electricity_loc['location'],
                                'input': (electricity_loc['database'], electricity_loc['code'])})
                    
            # Relink digital content activities:
            elif exc['name'] in [i['name'] for i in ds_to_regionalize] and exc['type'] != 'production':
                exc_loc = [ds_i for ds_i in ds_copy_list if ds_i['name'] == exc['name'] and ds_i['reference product'] == exc['product'] and ds_i['location'] == ds['location']][0]
                exc.update({'name': exc_loc['name'],
                            'location': exc_loc['location']})  

    return ds_copy_list

In [22]:
def link_exchanges(db, ei_db, biosphere_db):
    '''
    This function links technosphere exchanges within the database and/or to an external database
    and biosphere exchanges with the biosphere database (only unlinked exchanges).

    Returns a dictionary with the linked database
    '''
    db_linked = copy.deepcopy(db)
    
    production = lambda x: x["type"] == "production"  
    technosphere = lambda x: x["type"] == "technosphere"
    biosphere = lambda x: x["type"] == "biosphere"
    
    for ds in db_linked:
        
        for exc in filter(production, ds["exchanges"]):
            exc.update({'input': (ds['database'], ds['code'])})
        
        for exc in filter(technosphere, ds["exchanges"]):
            try:
                exc_link = wurst.get_one(db + ei_db,
                                         wurst.equals("name", exc['name']),
                                         wurst.equals("reference product", exc['product']),
                                         wurst.equals("location", exc['location'])
                                        )
                exc.update({'input': (exc_link['database'], exc_link['code'])})
            except Exception:
                print(exc['name'], exc['product'], exc['location'])
                raise
            
        for exc in filter(biosphere, ds["exchanges"]):
            if 'input' not in exc:
                try:
                    ef_code = [ef['code'] for ef in biosphere_db if ef['name'] == exc['name'] and 
                                                                    ef['unit'] == exc['unit'] and 
                                                                    ef['categories'] == exc['categories']][0]
                    exc.update({'input': ('biosphere3', ef_code)})   
                except Exception:
                    print(exc['name'], exc['unit'], exc['categories'])
                    raise

    return db_linked